In [15]:
import tensorflow as tf

import utils
import word2vec_utils

In [16]:
BATCH_SIZE = 128
VOCAB_SIZE = 50000
SKIP_WINDOW = 1 
VISUAL_FLD = 'visualization'

DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016
NUM_VISUALIZE = 3000        # number of tokens to visualize


In [17]:
def gen():
    yield from word2vec_utils.batch_gen(DOWNLOAD_URL, EXPECTED_BYTES, VOCAB_SIZE, 
                                        BATCH_SIZE, SKIP_WINDOW, VISUAL_FLD)

In [18]:
type([('UNK', -1)])

list

In [19]:
dataset = tf.data.Dataset.from_generator(gen, 
                                (tf.int32, tf.int32), 
                                (tf.TensorShape([BATCH_SIZE]), tf.TensorShape([BATCH_SIZE, 1])))

In [20]:
iterator = dataset.make_initializable_iterator()
center_words, target_words = iterator.get_next()

In [21]:
with tf.Session() as sess:
    sess.run(iterator.initializer)   
    t1=sess.run([center_words, target_words])
    print(t1)

data/text8.zip already exists
[array([ 5234,  3081,  3081,    12,    12,     6,     6,   195,   195,
           2,     2,  3134,  3134,    46,    46,    59,    59,   156,
         156,   128,   128,   742,   742,   477,   477, 10572, 10572,
         134,   134,     1,     1, 27350, 27350,     2,     2,     1,
           1,   103,   103,   855,   855,     3,     3,     1,     1,
       15068, 15068,     0,     0,     2,     2,     1,     1,   151,
         151,   855,   855,  3581,  3581,     1,     1,   195,   195,
          11,    11,   191,   191,    59,    59,     5,     5,     6,
           6, 10713, 10713,   215,   215,     7,     7,  1325,  1325,
         105,   105,   455,   455,    20,    20,    59,    59,  2732,
        2732,   363,   363,     7,     7,  3673,  3673,     1,     1,
         709,   709,     2,     2,   372,   372,    27,    27,    41,
          41,    37,    37,    54,    54,   540,   540,    98,    98,
          12,    12,     6,     6,  1424,  1424,  2758,  27

In [16]:
type(t1)

list

In [18]:
len(t1)

2

In [22]:
t1[0]

array([ 5234,  3081,  3081,    12,    12,     6,     6,   195,   195,
           2,     2,  3134,  3134,    46,    46,    59,    59,   156,
         156,   128,   128,   742,   742,   477,   477, 10572, 10572,
         134,   134,     1,     1, 27350, 27350,     2,     2,     1,
           1,   103,   103,   855,   855,     3,     3,     1,     1,
       15068, 15068,     0,     0,     2,     2,     1,     1,   151,
         151,   855,   855,  3581,  3581,     1,     1,   195,   195,
          11,    11,   191,   191,    59,    59,     5,     5,     6,
           6, 10713, 10713,   215,   215,     7,     7,  1325,  1325,
         105,   105,   455,   455,    20,    20,    59,    59,  2732,
        2732,   363,   363,     7,     7,  3673,  3673,     1,     1,
         709,   709,     2,     2,   372,   372,    27,    27,    41,
          41,    37,    37,    54,    54,   540,   540,    98,    98,
          12,    12,     6,     6,  1424,  1424,  2758,  2758,    19,
          19,   568]

In [24]:
t1[1]

array([[ 3081],
       [ 5234],
       [   12],
       [ 3081],
       [    6],
       [   12],
       [  195],
       [    6],
       [    2],
       [  195],
       [ 3134],
       [    2],
       [   46],
       [ 3134],
       [   59],
       [   46],
       [  156],
       [   59],
       [  128],
       [  156],
       [  742],
       [  128],
       [  477],
       [  742],
       [10572],
       [  477],
       [  134],
       [10572],
       [    1],
       [  134],
       [27350],
       [    1],
       [    2],
       [27350],
       [    1],
       [    2],
       [  103],
       [    1],
       [  855],
       [  103],
       [    3],
       [  855],
       [    1],
       [    3],
       [15068],
       [    1],
       [    0],
       [15068],
       [    2],
       [    0],
       [    1],
       [    2],
       [  151],
       [    1],
       [  855],
       [  151],
       [ 3581],
       [  855],
       [    1],
       [ 3581],
       [  195],
       [    1],
       [

In [24]:
import zipfile
def read_data(file_path):
    """ Read data into a list of tokens 
    There should be 17,005,207 tokens
    """
    with zipfile.ZipFile(file_path) as f:
        words = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return words

In [25]:
local_dest = 'data/text8.zip'
utils.download_one_file(DOWNLOAD_URL, local_dest, EXPECTED_BYTES)
words = read_data(local_dest)
print(type(words))

data/text8.zip already exists
<class 'list'>


### There are 17,005,207 tokens if you count tokens by splitting the text by blank space.

In [26]:
words[0]

'anarchism'

In [27]:
words[1]

'originated'

In [28]:
len(words)

17005207

In [30]:
import os
from collections import Counter
utils.safe_mkdir('visualization')
file = open(os.path.join('visualization', 'vocab.tsv'), 'w')
dictionary = dict()
count = [('UNK', -1)]
index = 0
count.extend(Counter(words).most_common(VOCAB_SIZE - 1))
print(len(count))
print(count[0])
print(count[1])
print(count[2])

50000
('UNK', -1)
('the', 1061396)
('of', 593677)


In [31]:
type(count)

list

In [32]:
for word, _ in count:
        dictionary[word] = index
        print(word)
        print(index)
        index += 1

UNK
0
the
1
of
2
and
3
one
4
in
5
a
6
to
7
zero
8
nine
9
two
10
is
11
as
12
eight
13
for
14
s
15
five
16
three
17
was
18
by
19
that
20
four
21
six
22
seven
23
with
24
on
25
are
26
it
27
from
28
or
29
his
30
an
31
be
32
this
33
which
34
at
35
he
36
also
37
not
38
have
39
were
40
has
41
but
42
other
43
their
44
its
45
first
46
they
47
some
48
had
49
all
50
more
51
most
52
can
53
been
54
such
55
many
56
who
57
new
58
used
59
there
60
after
61
when
62
into
63
american
64
time
65
these
66
only
67
see
68
may
69
than
70
world
71
i
72
b
73
would
74
d
75
no
76
however
77
between
78
about
79
over
80
years
81
states
82
people
83
war
84
during
85
united
86
known
87
if
88
called
89
use
90
th
91
system
92
often
93
state
94
so
95
history
96
will
97
up
98
while
99
where
100
city
101
being
102
english
103
then
104
any
105
both
106
under
107
out
108
made
109
well
110
her
111
e
112
number
113
government
114
them
115
m
116
later
117
since
118
him
119
part
120
name
121
c
122
century
123
through
124
because

apollo
1641
jazz
1642
helped
1643
hong
1644
slightly
1645
quantum
1646
runs
1647
hot
1648
presidential
1649
attempted
1650
boston
1651
asked
1652
kept
1653
fixed
1654
dictionary
1655
agreed
1656
plays
1657
bishop
1658
alphabet
1659
determined
1660
sweden
1661
buildings
1662
traditions
1663
reign
1664
existing
1665
communications
1666
violence
1667
ages
1668
phase
1669
hydrogen
1670
controversial
1671
meant
1672
formation
1673
twenty
1674
thousands
1675
agriculture
1676
resulted
1677
movies
1678
occurred
1679
focus
1680
offered
1681
estimated
1682
heavily
1683
discovery
1684
mi
1685
tour
1686
comics
1687
zone
1688
ring
1689
connection
1690
develop
1691
governments
1692
aspects
1693
argue
1694
damage
1695
austrian
1696
albert
1697
ibm
1698
factors
1699
avoid
1700
combination
1701
combat
1702
pro
1703
affairs
1704
novels
1705
sum
1706
expansion
1707
doctrine
1708
competition
1709
hungary
1710
leadership
1711
category
1712
flow
1713
week
1714
genetic
1715
soldiers
1716
leaving
1717
opened


welsh
3284
copper
3285
bases
3286
gibraltar
3287
jan
3288
survey
3289
ownership
3290
progressive
3291
advertising
3292
worth
3293
immediate
3294
statistical
3295
trained
3296
gallery
3297
commodore
3298
restricted
3299
integer
3300
senior
3301
slaves
3302
operator
3303
horror
3304
client
3305
hotel
3306
corps
3307
disputed
3308
translations
3309
empty
3310
relief
3311
fm
3312
stress
3313
anthony
3314
farm
3315
implemented
3316
involve
3317
keys
3318
sox
3319
esperanto
3320
psychological
3321
canon
3322
restrictions
3323
variables
3324
bureau
3325
criticized
3326
detail
3327
column
3328
uniform
3329
moses
3330
firm
3331
cleveland
3332
virus
3333
specialized
3334
libraries
3335
describing
3336
adapted
3337
drink
3338
outer
3339
ruling
3340
females
3341
offices
3342
send
3343
senator
3344
disney
3345
couple
3346
seconds
3347
reverse
3348
slavery
3349
biggest
3350
verse
3351
inch
3352
depicted
3353
colorado
3354
civilian
3355
conquered
3356
duty
3357
lose
3358
rocks
3359
calvin
3360
contra

wallace
4961
acceptable
4962
remember
4963
suspended
4964
reagan
4965
dealt
4966
discussions
4967
mccarthy
4968
fiscal
4969
observers
4970
documentation
4971
pilots
4972
passengers
4973
tanks
4974
deposits
4975
sheet
4976
weber
4977
honduras
4978
alberta
4979
enforcement
4980
yale
4981
vessel
4982
poe
4983
marxist
4984
explicit
4985
critique
4986
similarities
4987
dramatically
4988
initiated
4989
promised
4990
tense
4991
statute
4992
ions
4993
ba
4994
barrel
4995
galaxies
4996
edgar
4997
tribal
4998
trace
4999
corn
5000
dying
5001
assassinated
5002
wolf
5003
robin
5004
joan
5005
rhythm
5006
lutheran
5007
willing
5008
intention
5009
firing
5010
pleasure
5011
carefully
5012
famously
5013
faculty
5014
paradox
5015
criticisms
5016
nationalism
5017
designation
5018
pushed
5019
myths
5020
thesis
5021
modification
5022
craft
5023
es
5024
hemisphere
5025
boxing
5026
delivery
5027
resident
5028
extending
5029
feminist
5030
verbs
5031
daughters
5032
orleans
5033
lawyer
5034
arkansas
5035
jeffers

6676
acclaimed
6677
carthage
6678
closing
6679
mini
6680
isolation
6681
kit
6682
eg
6683
venezuela
6684
consumed
6685
armenia
6686
balkan
6687
allegiance
6688
modifications
6689
ff
6690
motorcycle
6691
rangle
6692
genuine
6693
anxiety
6694
labeled
6695
reprinted
6696
mask
6697
prone
6698
resolve
6699
domains
6700
macdonald
6701
questioned
6702
toll
6703
physicists
6704
cartoons
6705
freight
6706
ferry
6707
unrelated
6708
tribunal
6709
gorbachev
6710
cypriot
6711
scriptures
6712
viking
6713
ithaca
6714
sabbath
6715
compatibility
6716
profession
6717
attending
6718
masculine
6719
associates
6720
nerve
6721
beads
6722
optional
6723
cuts
6724
gibson
6725
conrad
6726
approximation
6727
polynomial
6728
wavelength
6729
armoured
6730
nice
6731
marxism
6732
couples
6733
motto
6734
probe
6735
devised
6736
fertile
6737
deliver
6738
zappa
6739
structured
6740
ecological
6741
fu
6742
carved
6743
architects
6744
strait
6745
dozen
6746
brigade
6747
scandinavian
6748
gospels
6749
lanka
6750
bankruptcy

...

41960
stabilise
41961
alde
41962
museu
41963
palomar
41964
generalitat
41965
affluence
41966
conceals
41967
babrak
41968
khalq
41969
dharmas
41970
bacharach
41971
agreeable
41972
backer
41973
cashier
41974
virginal
41975
prattica
41976
rameau
41977
yersinia
41978
pestis
41979
roslin
41980
landgrave
41981
sumatran
41982
biro
41983
petro
41984
distressing
41985
valproate
41986
unresponsive
41987
lazuli
41988
env
41989
oop
41990
carney
41991
gerda
41992
decrypted
41993
mishap
41994
hopewell
41995
stormbringer
41996
exhortation
41997
mikado
41998
sada
41999
bureaucracies
42000
nepotism
42001
sandhi
42002
brochs
42003
generalisations
42004
sitter
42005
dolmens
42006
docile
42007
ghibellines
42008
separators
42009
garrick
42010
ennedi
42011
insolvency
42012
jungingen
42013
fk
42014
vieux
42015
nicolau
42016
wolfhounds
42017
mohel
42018
circumcisions
42019
fiduciary
42020
harmonization
42021
waltham
42022
forgetfulness
42023
chosroes
42024
papier
42025
televangelist
42026
honorably
42027
narn

43528
officiate
43529
vagantes
43530
malleus
43531
cyclopes
43532
berle
43533
kaczynski
43534
rewind
43535
iiis
43536
upc
43537
corticosteroids
43538
ganglion
43539
secretes
43540
hasmonaean
43541
arsacid
43542
ayin
43543
mortuary
43544
andree
43545
coder
43546
billet
43547
versace
43548
pierson
43549
messiaen
43550
banisteriopsis
43551
aymara
43552
marlene
43553
entheogens
43554
antipodes
43555
ergot
43556
crowell
43557
preconditions
43558
hovered
43559
namath
43560
carew
43561
kovacs
43562
secretory
43563
vanquish
43564
minden
43565
discontinue
43566
bissette
43567
waid
43568
wachowski
43569
lakshmi
43570
ashram
43571
limes
43572
taverns
43573
checkpoints
43574
augmenting
43575
cedes
43576
nola
43577
gwynedd
43578
canby
43579
belatedly
43580
reinsurance
43581
analyte
43582
gunboats
43583
affaires
43584
schuman
43585
wallenberg
43586
transitioning
43587
sata
43588
accidently
43589
abhorrent
43590
mais
43591
spiracles
43592
somber
43593
pcd
43594
bax
43595
bak
43596
pathologies
43597
h

dagobert
45097
gard
45098
norah
45099
piotr
45100
methylated
45101
activator
45102
ethene
45103
esterification
45104
fished
45105
roofed
45106
cassady
45107
vern
45108
bogota
45109
hathaway
45110
botvinnik
45111
polgar
45112
qxd
45113
tarmac
45114
cogency
45115
massing
45116
sidebar
45117
marginalism
45118
ulrike
45119
privatizing
45120
purposefully
45121
amedeo
45122
ckel
45123
mitra
45124
agathon
45125
jahn
45126
impinging
45127
walras
45128
gangrene
45129
medlineplus
45130
cordell
45131
spiced
45132
oud
45133
connoisseur
45134
gasses
45135
unwitting
45136
zed
45137
chasm
45138
moralistic
45139
cameroun
45140
rature
45141
pers
45142
michaux
45143
downy
45144
nutty
45145
melito
45146
xenophobic
45147
jedwabne
45148
dhimma
45149
specter
45150
kristallnacht
45151
mapquest
45152
revamp
45153
lahr
45154
thereabouts
45155
sprengel
45156
academie
45157
vaux
45158
mooted
45159
coachella
45160
shoddy
45161
ramses
45162
blavatsky
45163
archivist
45164
newington
45165
diaconate
45166
exorbitant

46793
retells
46794
reuven
46795
molality
46796
dez
46797
summations
46798
topol
46799
microchips
46800
vivant
46801
abuser
46802
elamites
46803
surreptitiously
46804
glaser
46805
electroshock
46806
codata
46807
tilbury
46808
gv
46809
anesthetics
46810
articulations
46811
ulmanis
46812
lansana
46813
vivre
46814
sunspot
46815
llewelyn
46816
heathland
46817
mohiam
46818
adorning
46819
registrations
46820
rohde
46821
reissues
46822
spool
46823
xnor
46824
maxillofacial
46825
ziv
46826
rpp
46827
trending
46828
emigre
46829
pld
46830
wreckin
46831
detox
46832
travelogues
46833
jingle
46834
lhasa
46835
mtsho
46836
goalkeepers
46837
cavalcanti
46838
sacro
46839
sontag
46840
mosander
46841
lombardo
46842
skaro
46843
vigesimal
46844
boopsie
46845
zeke
46846
schottky
46847
cocalus
46848
isdb
46849
daemons
46850
singin
46851
translocation
46852
infraorder
46853
rnir
46854
hostname
46855
plunkett
46856
eamonn
46857
turbocharger
46858
dozenal
46859
dbv
46860
fujiko
46861
mailboxes
46862
wellcome
468

48567
keene
48568
cingular
48569
charlottenburg
48570
magisterial
48571
diligently
48572
unomig
48573
hurtful
48574
wissenschaft
48575
aggadah
48576
spotters
48577
duomo
48578
fanclub
48579
anxiolytic
48580
lugdunum
48581
sputtering
48582
infringes
48583
sacrum
48584
arco
48585
strat
48586
humbucker
48587
bearable
48588
neu
48589
nour
48590
wala
48591
talal
48592
hala
48593
unprofessional
48594
collages
48595
heijenoort
48596
zalta
48597
gheg
48598
kampen
48599
wheelers
48600
kahan
48601
instigate
48602
dov
48603
haaretz
48604
binyamin
48605
trygve
48606
offensively
48607
canceling
48608
vilified
48609
smuts
48610
vultures
48611
tessellations
48612
branko
48613
ayyash
48614
busting
48615
rq
48616
subterfuge
48617
exterminating
48618
rooftop
48619
dih
48620
vats
48621
virulence
48622
carniola
48623
trentino
48624
restatement
48625
amputee
48626
manukau
48627
earthworks
48628
supplanting
48629
trilled
48630
coed
48631
pantheist
48632
etta
48633
khanum
48634
retrofitted
48635
stoves
48636

In [33]:
dictionary.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 21

In [34]:
dictionary.keys()

dict_keys(['UNK', 'the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero', 'nine', 'two', 'is', 'as', 'eight', 'for', 's', 'five', 'three', 'was', 'by', 'that', 'four', 'six', 'seven', 'with', 'on', 'are', 'it', 'from', 'or', 'his', 'an', 'be', 'this', 'which', 'at', 'he', 'also', 'not', 'have', 'were', 'has', 'but', 'other', 'their', 'its', 'first', 'they', 'some', 'had', 'all', 'more', 'most', 'can', 'been', 'such', 'many', 'who', 'new', 'used', 'there', 'after', 'when', 'into', 'american', 'time', 'these', 'only', 'see', 'may', 'than', 'world', 'i', 'b', 'would', 'd', 'no', 'however', 'between', 'about', 'over', 'years', 'states', 'people', 'war', 'during', 'united', 'known', 'if', 'called', 'use', 'th', 'system', 'often', 'state', 'so', 'history', 'will', 'up', 'while', 'where', 'city', 'being', 'english', 'then', 'any', 'both', 'under', 'out', 'made', 'well', 'her', 'e', 'number', 'government', 'them', 'm', 'later', 'since', 'him', 'part', 'name', 'c', 'century', 'through', 'because', 

In [46]:
len(dictionary)

50000

In [47]:
len(words)

17005207

In [35]:
def convert_words_to_index(words, dictionary):
    """ Replace each word in the dataset with its index in the dictionary """
    return [dictionary[word] if word in dictionary else 0 for word in words]

In [48]:
words[0]

'anarchism'

In [49]:
'anarchism' in dictionary

True

In [50]:
dictionary['anarchism']

5234

In [51]:
index_words[0]

5234

In [44]:
'half' in dictionary

True

In [45]:
dictionary['half']

517

In [36]:
 index_words = convert_words_to_index(words, dictionary)

In [37]:
type(index_words)

list

In [38]:
len(index_words)

17005207

In [39]:
index_words[0]

5234

In [40]:
index_words[1]

3081

In [42]:
words[5234]

'so'

In [43]:
dictionary['so']

95